In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
import os
# pages = [f'https://www.aspca.org/adopt-pet/adoptable-dogs-your-local-shelter#petfocus_0=&page_0={page}&breed_0=&sex_0=&distance_0=100000&location_0=95476&action_0=search']


/Users/adrianabrazon/Documents/IronHack/final_project/.venv/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
#### WARNING : DO NOT RUN THIS CELL ##

######              LAST VERSION            ##########

# Lists to store extracted data
dog_pic_urls = []
dog_names = []
url_sources = []

# Loop to iterate through the pages

# for urls in range(0, 11):
urls = [f"https://www.aspca.org/adopt-pet/adoptable-dogs-your-local-shelter#petfocus_0=&page_0={pag}&breed_0=&sex_0=&distance_0=100000&location_0=95476&action_0=search" for pag in range(1,834)]

# Number of pages to process before saving data
pages_to_save = 80

# Directory path to save CSV files
save_directory = '/Volumes/Update/final_project/dog_files'

for url in urls:
    driver = webdriver.Chrome()
    driver.get(url)
            
    time.sleep(5)  # Adding a 5-second delay
        
    elements = driver.find_elements(By.CLASS_NAME, "rgtkSearchResultsCell")
    
    for element in elements:

        try:
            # Find the img tag for dog picture
            pic_url = element.find_element(By.CLASS_NAME, "rgtkSearchPetPicImg").get_attribute('src')
        except NoSuchElementException:
            pic_url = 'no photo'
                    
        dog_pic_urls.append(pic_url)

        dog_name = element.find_elements(By.TAG_NAME, "a")[1].text.strip()
        dog_names.append(dog_name)
            
        url_source = element.find_elements(By.TAG_NAME, "a")[1].get_attribute('onclick')
        url_sources.append(url_source)

    df = pd.DataFrame({'doggo_name' : dog_names, 'doggo_pic_url': dog_pic_urls, 'url_source': url_sources})

    #ensure that we clean duplicates
    df = df.drop_duplicates().reset_index(drop=True)

        # Quit the WebDriver
    driver.quit()

for idx, pag in enumerate(urls):
    if (idx + 1) % pages_to_save == 0 or idx == len(urls) - 1:
        # Save data to CSV file
        csv_file_path = os.path.join(save_directory, f'dog_data_page_{idx}.csv')
        df.to_csv(csv_file_path, index=False)


In [3]:
df = pd.read_csv('dog_files/dog_data_webscrape.csv')

In [4]:
df

,doggo_name,doggo_pic_url,url_source
0,Keisha,https://cdn.rescuegroups.org/4319/pictures/ani...,"toolkitFocusPet_0(0, 19921455, 0);return false;"
1,Mayflower,https://cdn.rescuegroups.org/4319/pictures/ani...,"toolkitFocusPet_0(0, 20026683, 1);return false;"
2,George,https://cdn.rescuegroups.org/4319/pictures/ani...,"toolkitFocusPet_0(0, 19628021, 2);return false;"
3,Billie,https://cdn.rescuegroups.org/4319/pictures/ani...,"toolkitFocusPet_0(0, 20109890, 3);return false;"
4,Henry,https://cdn.rescuegroups.org/4319/pictures/ani...,"toolkitFocusPet_0(0, 20109891, 4);return false;"
...,...,...,...
19987,JC Penny,https://cdn.rescuegroups.org/4598/pictures/ani...,"toolkitFocusPet_0(0, 19613415, 19987);return f..."
19988,Nate,no photo,"toolkitFocusPet_0(0, 19627552, 19988);return f..."
19989,Rocket,https://cdn.rescuegroups.org/4598/pictures/ani...,"toolkitFocusPet_0(0, 19628000, 19989);return f..."
19990,Hunter,https://cdn.rescuegroups.org/4598/pictures/ani...,"toolkitFocusPet_0(0, 19642640, 19990);return f..."


In [44]:
df.tail()

,doggo_name,doggo_pic_url,url_source
19987,JC Penny,https://cdn.rescuegroups.org/4598/pictures/ani...,"toolkitFocusPet_0(0, 19613415, 19987);return f..."
19988,Nate,no photo,"toolkitFocusPet_0(0, 19627552, 19988);return f..."
19989,Rocket,https://cdn.rescuegroups.org/4598/pictures/ani...,"toolkitFocusPet_0(0, 19628000, 19989);return f..."
19990,Hunter,https://cdn.rescuegroups.org/4598/pictures/ani...,"toolkitFocusPet_0(0, 19642640, 19990);return f..."
19991,Beau,https://cdn.rescuegroups.org/4598/pictures/ani...,"toolkitFocusPet_0(0, 19649232, 19991);return f..."


In [5]:
# 473 dogs with no picture
(df['doggo_pic_url']=='no photo').value_counts()


doggo_pic_url
False    19519
True       473
Name: count, dtype: int64

In [46]:
# now we build the doggo_url
# url format =  https://www.aspca.org/adopt-pet/adoptable-dogs-your-local-shelter#action_0=pet&animalID_0={animal_id}&petIndex_0={pet_index}

# we create the empty list to store our results
doggo_url = []
animal_ids = []
for i in range(len(df)):
    animal_id = df.url_source[i].split(',')[1].strip()
    doggy_url = f"https://www.aspca.org/adopt-pet/adoptable-dogs-your-local-shelter#action_0=pet&animalID_0={animal_id}&petIndex_0={i}"
    doggo_url.append(doggy_url)
    animal_ids.append(animal_id)

In [47]:
# we now replace url_source values with the doggo_url list that we just populated
# and add a new 'animal_id' column 
df['url_source'] = doggo_url
df = df.rename(columns={'url_source':'doggo_url'})
df['animal_id'] = animal_ids

In [50]:
df.head()

,doggo_name,doggo_pic_url,doggo_url,animal_id
0,Keisha,https://cdn.rescuegroups.org/4319/pictures/ani...,https://www.aspca.org/adopt-pet/adoptable-dogs...,19921455
1,Mayflower,https://cdn.rescuegroups.org/4319/pictures/ani...,https://www.aspca.org/adopt-pet/adoptable-dogs...,20026683
2,George,https://cdn.rescuegroups.org/4319/pictures/ani...,https://www.aspca.org/adopt-pet/adoptable-dogs...,19628021
3,Billie,https://cdn.rescuegroups.org/4319/pictures/ani...,https://www.aspca.org/adopt-pet/adoptable-dogs...,20109890
4,Henry,https://cdn.rescuegroups.org/4319/pictures/ani...,https://www.aspca.org/adopt-pet/adoptable-dogs...,20109891


In [53]:
# we will drop this row........
df = df.drop(df[df['doggo_name'] == 'Braedon & Brighton - Rest In Peace'].index).reset_index(drop=True)

In [54]:
df

,doggo_name,doggo_pic_url,doggo_url,animal_id
0,Keisha,https://cdn.rescuegroups.org/4319/pictures/ani...,https://www.aspca.org/adopt-pet/adoptable-dogs...,19921455
1,Mayflower,https://cdn.rescuegroups.org/4319/pictures/ani...,https://www.aspca.org/adopt-pet/adoptable-dogs...,20026683
2,George,https://cdn.rescuegroups.org/4319/pictures/ani...,https://www.aspca.org/adopt-pet/adoptable-dogs...,19628021
3,Billie,https://cdn.rescuegroups.org/4319/pictures/ani...,https://www.aspca.org/adopt-pet/adoptable-dogs...,20109890
4,Henry,https://cdn.rescuegroups.org/4319/pictures/ani...,https://www.aspca.org/adopt-pet/adoptable-dogs...,20109891
...,...,...,...,...
19986,JC Penny,https://cdn.rescuegroups.org/4598/pictures/ani...,https://www.aspca.org/adopt-pet/adoptable-dogs...,19613415
19987,Nate,no photo,https://www.aspca.org/adopt-pet/adoptable-dogs...,19627552
19988,Rocket,https://cdn.rescuegroups.org/4598/pictures/ani...,https://www.aspca.org/adopt-pet/adoptable-dogs...,19628000
19989,Hunter,https://cdn.rescuegroups.org/4598/pictures/ani...,https://www.aspca.org/adopt-pet/adoptable-dogs...,19642640


In [55]:
# testing the animal_id matches the pet
df.doggo_pic_url[0]

'https://cdn.rescuegroups.org/4319/pictures/animals/19921/19921455/96951790.jpg'

In [56]:
# Finally we save our DataFrame as a CSV file!
save_directory1 = '/Users/adrianabrazon/Documents/IronHack/final_project'
csv_file_path = os.path.join(save_directory1, 'pets_for_adoption.csv')
df.to_csv(csv_file_path, index=False)